# EuroCup 2024 Here We Go

### EuroCup Logo

![EuroTiesheet](/files/tables/eurobw.jpg)

### Playoffs Consideration

- Path A: Wales/Finland vs Poland/Estonia (Poland)
- Path B: Bosnia and Herzegovina/Ukraine vs Israel/Iceland (Ukraine)
- Path C: Georgia/Luxembourg vs Greece/Kazakhstan (Greece)

## Adding All team to teams table

In [0]:
%sql
-- Create or replace the table with the OVERWRITE option
CREATE OR REPLACE TABLE allteams (
  TeamId INTEGER,
  TeamName VARCHAR(255),
  TeamGroup VARCHAR(255)
);
-- Insert the provided data into the "allteams" table
INSERT INTO allteams (TeamId, TeamName, TeamGroup)
VALUES
  (1, 'Germany', 'A'),
  (2, 'Scotland', 'A'),
  (3, 'Hungary', 'A'),
  (4, 'Switzerland', 'A'),
  (5, 'Spain', 'B'),
  (6, 'Croatia', 'B'),
  (7, 'Italy', 'B'),
  (8, 'Albania', 'B'),
  (9, 'Slovenia', 'C'),
  (10, 'Denmark', 'C'),
  (11, 'Serbia', 'C'),
  (12, 'England', 'C'),
  (13, 'Poland', 'D'),
  (14, 'Netherlands', 'D'),
  (15, 'Austria', 'D'),
  (16, 'France', 'D'),
  (17, 'Belgium', 'E'),
  (18, 'Slovakia', 'E'),
  (19, 'Romania', 'E'),
  (20, 'Ukraine', 'E'),
  (21, 'Türkiye', 'F'),
  (22, 'Greece', 'F'),
  (23, 'Portugal', 'F'),
  (24, 'Czech', 'F');

-- Verify the data in the "allteams" table
SELECT * FROM allteams;


TeamId,TeamName,TeamGroup
1,Germany,A
2,Scotland,A
3,Hungary,A
4,Switzerland,A
5,Spain,B
6,Croatia,B
7,Italy,B
8,Albania,B
9,Slovenia,C
10,Denmark,C


rm -r dbfs:/user/hive/warehouse/ranking_group

### EuroTiesheet

![EuroTiesheet](/files/tables/EuroTiesheet.png)

### Total Combination for third palce

![EuroTiesheet](/files/tables/comb.png)

## Creating table for the above combination 
-The specific match-ups involving the third-placed teams depend on which four third-placed teams qualify for the round of 16

In [0]:
%sql
CREATE TABLE THIRDPLACECOMBO (
  ID INTEGER,
  SELECTEDGROUPS VARCHAR(4),
  Team1B CHAR(1),
  Team1C CHAR(1),
  Team1E CHAR(1),
  Team1F CHAR(1)
);

-- Step 2: Insert the provided values into the "THIRDMATCHUP" table
INSERT INTO THIRDPLACECOMBO (ID, SELECTEDGROUPS, Team1B, Team1C, Team1E, Team1F) VALUES
(1, 'ABCD', 'A', 'D', 'B', 'C'),
(2, 'ABCE', 'A', 'E', 'B', 'C'),
(3, 'ABCF', 'A', 'F', 'B', 'C'),
(4, 'ABDE', 'D', 'E', 'A', 'B'),
(5, 'ABDF', 'D', 'F', 'A', 'B'),
(6, 'ABEF', 'E', 'F', 'B', 'A'),
(7, 'ACDE', 'E', 'D', 'C', 'A'),
(8, 'ACDF', 'F', 'D', 'C', 'A'),
(9, 'ACEF', 'E', 'F', 'C', 'A'),
(10, 'ADEF', 'E', 'F', 'D', 'A'),
(11, 'BCDE', 'E', 'D', 'B', 'C'),
(12, 'BCDF', 'F', 'D', 'C', 'B'),
(13, 'BCEF', 'F', 'E', 'C', 'B'),
(14, 'BDEF', 'F', 'E', 'D', 'B'),
(15, 'CDEF', 'F', 'E', 'D', 'C');


select * from THIRDPLACECOMBO


ID,SELECTEDGROUPS,Team1B,Team1C,Team1E,Team1F
1,ABCD,A,D,B,C
2,ABCE,A,E,B,C
3,ABCF,A,F,B,C
4,ABDE,D,E,A,B
5,ABDF,D,F,A,B
6,ABEF,E,F,B,A
7,ACDE,E,D,C,A
8,ACDF,F,D,C,A
9,ACEF,E,F,C,A
10,ADEF,E,F,D,A


### Playing the Group stage matchies and insert into matchesofcountry table

- Define matchesofcountry talbe with required columns to store match outcomes
- each team in each group plays with all the other team in the same group
- set the score to random number between 1 to 6
- update the Result column on the basics of match score, if team1 score and team2 score are same then set Result to draw


In [0]:
%sql
CREATE TABLE matchesofcountry (
  MatchId INTEGER,
  Team1 VARCHAR(255),
  Team2 VARCHAR(255),
  Team1Score INTEGER,
  Team2Score INTEGER,
  Result VARCHAR(255),
  Stage VARCHAR(255)
);

INSERT INTO matchesofcountry (MatchId, Team1, Team2, Team1Score, Team2Score, Result, Stage)
SELECT DISTINCT
    ROW_NUMBER() OVER (ORDER BY t1.TeamId) AS MatchId,
    t1.TeamName AS Team1,
    t2.TeamName AS Team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' AS Result,
    'group stage' AS Stage
FROM
    allteams t1
JOIN
    allteams t2 ON t1.TeamGroup = t2.TeamGroup AND t1.TeamId < t2.TeamId;


-- Step 4: Calculate match results and update the "Result" column
UPDATE matchesofcountry
SET Result = 
    CASE
        WHEN Team1Score > Team2Score THEN Team1
        WHEN Team1Score < Team2Score THEN Team2
        ELSE 'draw'
    END;


-- Step 5: Verify the match results
SELECT * FROM matchesofcountry
order by MatchId;


MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage
1,Germany,Switzerland,6,4,Germany,group stage
2,Germany,Hungary,2,6,Hungary,group stage
3,Germany,Scotland,4,6,Scotland,group stage
4,Scotland,Switzerland,5,2,Scotland,group stage
5,Scotland,Hungary,6,1,Scotland,group stage
6,Hungary,Switzerland,1,3,Switzerland,group stage
7,Spain,Albania,4,3,Spain,group stage
8,Spain,Italy,5,1,Spain,group stage
9,Spain,Croatia,3,3,draw,group stage
10,Croatia,Albania,5,2,Croatia,group stage


### Ranking Table after group stage matches 
- The four Third place teams to move on are determined by the follwoing criteria established by UEFA in the following specific order
- 1. most point
- 2. goal difference
- 3. most goals scored
- 4. most wins
- 5. fewest disciplinary points in group stage matches(not consider in this porject)
- 6. position in the overall European Qualifiers ranking(not consider in this porject)

##### How I create ranking_group table
- First define the required column
- create MatchResults cte with columns TeamName, win(to calculate the total number of wins), points(to calculate total point), gaolscored and golaconcede(to calculate goal difference)
- now insert the values to the ranking_group using allteams table and MatchResults cte. Firstly group by teamid to calculate the total number of wins, points and goaldifference form MatchResult Cte, then group by teamgroup to calculate the ranking for each team with in the groups
- on the basics of inserted ranking_groups compute the Rankings columns. To use ranking column use row_number() since if all the rank determining creteria are same it provide the random rank 
- 1st and 2nd teams from each groups will be directly qualified for Round of 16 contributing the 12 outof 16 teams. Remaining 4 teams will be selected as Top 4 teams among all 6 teams which finish 3rd in their respective groups.
- create TopRank3Teams cte to find top 4 team which are 3rd in each group. create IsProgressCalc cte to set the isprogress to true for the 1st, 2nd and top 4 3rd place teams and finally merge to the target table i.e ranking_group

In [0]:
%sql
-- Step 1: Create the "ranking" table
CREATE TABLE ranking_group (
  TeamId INTEGER,
  TeamName VARCHAR(255),
  TeamGroup VARCHAR(255),
  Points INTEGER,
  Rankings INTEGER,
  GoalsScored INTEGER,
  GoalsConceded INTEGER,
  GoalDifference INTEGER,
  WinCount INTEGER, -- New column for win count
  IsProgress BOOLEAN
);

-- Calculate points, goals scored, goals conceded, goal difference, and win count for each team
WITH MatchResults AS (
  SELECT 
    Team1 AS TeamName,
    CASE 
      WHEN Result = CAST(Team1 AS VARCHAR(255)) THEN 1  -- Team1 wins
      ELSE 0  -- Team1 does not win
    END AS Win, 
    CASE 
      WHEN Result = CAST(Team1 AS VARCHAR(255)) THEN 3  -- Team1 wins
      WHEN Result = CAST(Team2 AS VARCHAR(255)) THEN 0  -- Team1 loses
      ELSE 1  -- Draw
    END AS Points,
    Team1Score AS GoalsScored, -- Goals scored by Team1
    Team2Score AS GoalsConceded -- Goals conceded by Team1
  FROM matchesofcountry
  UNION ALL
  SELECT 
    Team2 AS TeamName,
    CASE 
      WHEN Result = CAST(Team2 AS VARCHAR(255)) THEN 1  -- Team2 wins
      ELSE 0  -- Team2 does not win
    END AS Win, 
    CASE 
      WHEN Result = CAST(Team2 AS VARCHAR(255)) THEN 3  -- Team2 wins
      WHEN Result = CAST(Team1 AS VARCHAR(255)) THEN 0  -- Team2 loses
      ELSE 1  -- Draw
    END AS Points,
    Team2Score AS GoalsScored, -- Goals scored by Team2
    Team1Score AS GoalsConceded -- Goals conceded by Team2
  FROM matchesofcountry
)
INSERT INTO ranking_group (TeamId, TeamName, TeamGroup, Points, Rankings, GoalsScored, GoalsConceded, GoalDifference, WinCount, IsProgress)
SELECT
  t.TeamId,
  t.TeamName,
  t.TeamGroup,
  SUM(M.Points) AS Points,
  0 as Rankings,
  SUM(M.GoalsScored) AS GoalsScored, -- Total goals scored by the team
  SUM(M.GoalsConceded) AS GoalsConceded, -- Total goals conceded by the team
  SUM(M.GoalsScored) - SUM(M.GoalsConceded) AS GoalDifference, -- Goal difference for the team
  SUM(M.Win) AS WinCount,
  0 as IsProgress -- Total number of wins for the team
FROM 
  allteams t
LEFT JOIN
  MatchResults M ON t.TeamName = M.TeamName
GROUP BY
  t.TeamId, t.TeamName, t.TeamGroup;

-- Rank allteams within each group based on points, goal difference, goals scored, and win count
WITH RankedTeams AS (
  SELECT 
    TeamName,
    row_number() OVER (PARTITION BY TeamGroup ORDER BY Points DESC, GoalDifference DESC, GoalsScored DESC, WinCount DESC) AS GroupRank
  FROM ranking_group
)
MERGE INTO ranking_group r
USING RankedTeams rt
ON r.TeamName = rt.TeamName
WHEN MATCHED THEN UPDATE SET r.Rankings = rt.GroupRank;


--creating a ctc column to calculate the isprogress column and mearsing to the ranking table
WITH RankedTeams AS (
  SELECT 
    *,
    row_number() OVER (PARTITION BY TeamGroup ORDER BY Points DESC, GoalDifference DESC, GoalsScored DESC, WinCount DESC) AS GroupRank
  FROM ranking_group
),
TopRank3Teams AS (
    SELECT 
    *,
    row_number() OVER (ORDER BY Points DESC, GoalDifference DESC, GoalsScored DESC, WinCount DESC) AS RankWithinGroup
  FROM ranking_group
  WHERE Rankings = 3
),
IsProgressCalc AS (
  SELECT 
    r.*,
    CASE 
        WHEN r.GroupRank = 4 THEN 0
        WHEN r.GroupRank = 3 AND t.RankWithinGroup <= 4 THEN 1
        When r.groupRank = 1 or r.grouprank = 2 then 1
        else 0
    END AS IsProgressCalc 
  FROM RankedTeams r
  LEFT JOIN TopRank3Teams t ON r.TeamName = t.TeamName
)

MERGE INTO ranking_group r
USING IsProgressCalc ipc
ON r.TeamId = ipc.TeamId
WHEN MATCHED THEN UPDATE SET r.isprogress = ipc.IsProgressCalc;
SELECT * FROM ranking_group
order by TeamGroup, Rankings;


TeamId,TeamName,TeamGroup,Points,Rankings,GoalsScored,GoalsConceded,GoalDifference,WinCount,IsProgress
2,Scotland,A,9,1,17,7,10,3,true
4,Switzerland,A,3,2,9,12,-3,1,true
3,Hungary,A,3,3,8,11,-3,1,false
1,Germany,A,3,4,12,16,-4,1,false
5,Spain,B,7,1,12,7,5,2,true
6,Croatia,B,7,2,14,10,4,2,true
8,Albania,B,3,3,9,12,-3,1,false
7,Italy,B,0,4,9,15,-6,0,false
10,Denmark,C,6,1,9,8,1,2,true
11,Serbia,C,4,2,10,7,3,1,true


### Playing Roundof 16 matches
- First determine the among 15 combination for possible 4 3rd place teams, which combination was selected. For this use rankings = 3 and is progress = true and concate the selected groups in alphabetic order.
- After the selected groups calculate, find the matching row in THIRDPLACECOMBO table as SelectedGrpTbl
- Now, matchid upto 36 are group stage matches, and ids from 36 to 44 are for roundof 16 matches since there are 8 matches in round of 16
- add the round of 16 matches to the matchesofcountry table on the basics of tie sheet and Third place combination table
- add random score form 1-6 to each team for each match
- based on the score update the result column for roundof 16

In [0]:
%sql

With SelectedGroup AS
(
SELECT 
    CONCAT_WS('', SORT_ARRAY(COLLECT_SET(SUBSTRING(TeamGroup, 1, 1)))) AS Result
FROM ranking_group
WHERE Rankings = 3 AND IsProgress = TRUE
),

SelectedGrpTbl AS
(
SELECT * 
FROM thirdplacecombo
WHERE thirdplacecombo.SELECTEDGROUPS = (SELECT Result FROM SelectedGroup)
)
INSERT INTO matchesofcountry (MatchId, team1, team2, team1score, team2score, Result, stage)
SELECT 
    37 AS MatchId, 
    (SELECT TeamName FROM ranking_group WHERE Rankings = 1 AND TeamGroup = 'B') AS team1, 
    (SELECT TeamName FROM ranking_group WHERE Rankings = 3 AND TeamGroup = (select Team1B from SelectedGrpTbl)) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Roundof 16' as stage
union All
SELECT 
    38 AS MatchId, 
    (SELECT TeamName FROM ranking_group WHERE Rankings = 1 AND TeamGroup = 'A') AS team1, 
    (SELECT TeamName FROM ranking_group WHERE Rankings = 2 AND TeamGroup = 'C') AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Roundof 16' as stage
UNION ALL
SELECT 
    39 AS MatchId, 
    (SELECT TeamName FROM ranking_group WHERE Rankings = 1 AND TeamGroup = 'F') AS team1, 
    (SELECT TeamName FROM ranking_group WHERE Rankings = 3 AND TeamGroup = (select Team1F from SelectedGrpTbl)) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Roundof 16' as stage
union All
SELECT 
    40 AS MatchId, 
    (SELECT TeamName FROM ranking_group WHERE Rankings = 2 AND TeamGroup = 'D') AS team1, 
    (SELECT TeamName FROM ranking_group WHERE Rankings = 2 AND TeamGroup = 'E') AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Roundof 16' as stage
UNION ALL
SELECT 
    41 AS MatchId, 
    (SELECT TeamName FROM ranking_group WHERE Rankings = 1 AND TeamGroup = 'E') AS team1, 
    (SELECT TeamName FROM ranking_group WHERE Rankings = 3 AND TeamGroup = (select Team1E from SelectedGrpTbl)) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Roundof 16' as stage
union All
SELECT 
    42 AS MatchId, 
    (SELECT TeamName FROM ranking_group WHERE Rankings = 1 AND TeamGroup = 'D') AS team1, 
    (SELECT TeamName FROM ranking_group WHERE Rankings = 2 AND TeamGroup = 'F') AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Roundof 16' as stage
UNION ALL
SELECT 
    43 AS MatchId, 
    (SELECT TeamName FROM ranking_group WHERE Rankings = 1 AND TeamGroup = 'C') AS team1, 
    (SELECT TeamName FROM ranking_group WHERE Rankings = 3 AND TeamGroup = (select Team1C from SelectedGrpTbl)) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Roundof 16' as stage
union All
SELECT 
    44 AS MatchId, 
    (SELECT TeamName FROM ranking_group WHERE Rankings = 2 AND TeamGroup = 'A') AS team1, 
    (SELECT TeamName FROM ranking_group WHERE Rankings = 2 AND TeamGroup = 'B') AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Roundof 16' as stage;
--updationg the Result column
UPDATE matchesofcountry
SET Result = 
    CASE 
        WHEN team1score > team2score THEN team1
        WHEN team1score < team2score THEN team2
        ELSE 
            CASE 
                WHEN (team1score + 1) % 2 = 1 THEN team1
                ELSE team2
            END
    END
where stage = 'Roundof 16';
SELECT * FROM matchesofcountry
order by MatchId;

MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage
1,Germany,Switzerland,6,4,Germany,group stage
2,Germany,Hungary,2,6,Hungary,group stage
3,Germany,Scotland,4,6,Scotland,group stage
4,Scotland,Switzerland,5,2,Scotland,group stage
5,Scotland,Hungary,6,1,Scotland,group stage
6,Hungary,Switzerland,1,3,Switzerland,group stage
7,Spain,Albania,4,3,Spain,group stage
8,Spain,Italy,5,1,Spain,group stage
9,Spain,Croatia,3,3,draw,group stage
10,Croatia,Albania,5,2,Croatia,group stage


### Quater Final Matches
- There are 4 matches for quater final i.e matchId from 45 to 48
- On the basics of result of roundof 16 matches matchups the quater final and update the result table on the basics of score

In [0]:
%sql
INSERT INTO matchesofcountry (MatchId, team1, team2, team1score, team2score, Result, stage)
SELECT 
    45 AS MatchId, 
    (SELECT Result FROM matchesofcountry WHERE MatchId = 37) AS team1, 
    (SELECT Result FROM matchesofcountry WHERE MatchId = 38) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Quater Final' as stage
UNION ALL
SELECT 
    46 AS MatchId, 
    (SELECT Result FROM matchesofcountry WHERE MatchId = 39) AS team1, 
    (SELECT Result FROM matchesofcountry WHERE MatchId = 40) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Quater Final' as stage
UNION ALL
SELECT 
    47 AS MatchId, 
    (SELECT Result FROM matchesofcountry WHERE MatchId = 41) AS team1, 
    (SELECT Result FROM matchesofcountry WHERE MatchId = 42) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Quater Final' as stage
UNION ALL
SELECT 
    48 AS MatchId, 
    (SELECT Result FROM matchesofcountry WHERE MatchId = 43) AS team1, 
    (SELECT Result FROM matchesofcountry WHERE MatchId = 44) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Quater Final' as stage;

UPDATE matchesofcountry
SET Result = 
    CASE 
        WHEN team1score > team2score THEN team1
        WHEN team1score < team2score THEN team2
        ELSE 
            CASE 
                WHEN (team1score + 1) % 2 = 1 THEN team1
                ELSE team2
            END
    END
where stage = 'Quater Final';

select * from matchesofcountry
order by MatchId

MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage
1,Germany,Switzerland,6,4,Germany,group stage
2,Germany,Hungary,2,6,Hungary,group stage
3,Germany,Scotland,4,6,Scotland,group stage
4,Scotland,Switzerland,5,2,Scotland,group stage
5,Scotland,Hungary,6,1,Scotland,group stage
6,Hungary,Switzerland,1,3,Switzerland,group stage
7,Spain,Albania,4,3,Spain,group stage
8,Spain,Italy,5,1,Spain,group stage
9,Spain,Croatia,3,3,draw,group stage
10,Croatia,Albania,5,2,Croatia,group stage


### Semi Final
- populate the semifinal matches on the basics of result of quater final and update the result column

In [0]:
%sql
INSERT INTO matchesofcountry (MatchId, team1, team2, team1score, team2score, Result, stage)
SELECT 
    49 AS MatchId, 
    (SELECT Result FROM matchesofcountry WHERE MatchId = 45) AS team1, 
    (SELECT Result FROM matchesofcountry WHERE MatchId = 46) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Semi Final' as stage
UNION ALL
SELECT 
    50 AS MatchId, 
    (SELECT Result FROM matchesofcountry WHERE MatchId = 47) AS team1, 
    (SELECT Result FROM matchesofcountry WHERE MatchId = 48) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Semi Final' as stage;

UPDATE matchesofcountry
SET Result = 
    CASE 
        WHEN team1score > team2score THEN team1
        WHEN team1score < team2score THEN team2
        ELSE 
            CASE 
                WHEN (team1score + 1) % 2 = 1 THEN team1
                ELSE team2
            END
    END
where stage = 'Semi Final';

select * from matchesofcountry
order by MatchId

MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage
1,Germany,Switzerland,6,4,Germany,group stage
2,Germany,Hungary,2,6,Hungary,group stage
3,Germany,Scotland,4,6,Scotland,group stage
4,Scotland,Switzerland,5,2,Scotland,group stage
5,Scotland,Hungary,6,1,Scotland,group stage
6,Hungary,Switzerland,1,3,Switzerland,group stage
7,Spain,Albania,4,3,Spain,group stage
8,Spain,Italy,5,1,Spain,group stage
9,Spain,Croatia,3,3,draw,group stage
10,Croatia,Albania,5,2,Croatia,group stage


In [0]:
%sql
SELECT Result FROM matchesofcountry WHERE stage='Quater Final' and Result not in (select Result FROM matchesofcountry WHERE stage='Semi Final') LIMIT 1 OFFSET 0

Result
Croatia


### Third place Game

In [0]:
%sql
INSERT INTO matchesofcountry (MatchId, team1, team2, team1score, team2score, Result, stage)
SELECT 
    51 AS MatchId, 
    (SELECT Result FROM matchesofcountry WHERE stage='Quater Final' and Result not in (select Result FROM matchesofcountry WHERE stage='Semi Final') LIMIT 1 OFFSET 0) AS team1, 
    (SELECT Result FROM matchesofcountry WHERE stage='Quater Final' and Result not in (select Result FROM matchesofcountry WHERE stage='Semi Final') LIMIT 1 OFFSET 1) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Third Place' as stage;
UPDATE matchesofcountry
SET Result = 
    CASE 
        WHEN team1score > team2score THEN team1
        WHEN team1score < team2score THEN team2
        ELSE 
            CASE 
                WHEN (team1score + 1) % 2 = 1 THEN team1
                ELSE team2
            END
    END
where stage = 'Third Place';

select * from matchesofcountry
order by MatchId

MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage
1,Germany,Switzerland,6,4,Germany,group stage
2,Germany,Hungary,2,6,Hungary,group stage
3,Germany,Scotland,4,6,Scotland,group stage
4,Scotland,Switzerland,5,2,Scotland,group stage
5,Scotland,Hungary,6,1,Scotland,group stage
6,Hungary,Switzerland,1,3,Switzerland,group stage
7,Spain,Albania,4,3,Spain,group stage
8,Spain,Italy,5,1,Spain,group stage
9,Spain,Croatia,3,3,draw,group stage
10,Croatia,Albania,5,2,Croatia,group stage


### Final at Munich, Olympiastadion Berlin 7/14 21:00 kickof

In [0]:
%sql
INSERT INTO matchesofcountry (MatchId, team1, team2, team1score, team2score, Result, stage)
SELECT 
    52 AS MatchId, 
    (SELECT Result FROM matchesofcountry WHERE MatchId = 49) AS team1, 
    (SELECT Result FROM matchesofcountry WHERE MatchId = 50) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Final' as stage;
UPDATE matchesofcountry
SET Result = 
    CASE 
        WHEN team1score > team2score THEN team1
        WHEN team1score < team2score THEN team2
        ELSE 
            CASE 
                WHEN (team1score + 1) % 2 = 1 THEN team1
                ELSE team2
            END
    END
where stage = 'Final';

select * from matchesofcountry
order by MatchId

MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage
1,Germany,Switzerland,6,4,Germany,group stage
2,Germany,Hungary,2,6,Hungary,group stage
3,Germany,Scotland,4,6,Scotland,group stage
4,Scotland,Switzerland,5,2,Scotland,group stage
5,Scotland,Hungary,6,1,Scotland,group stage
6,Hungary,Switzerland,1,3,Switzerland,group stage
7,Spain,Albania,4,3,Spain,group stage
8,Spain,Italy,5,1,Spain,group stage
9,Spain,Croatia,3,3,draw,group stage
10,Croatia,Albania,5,2,Croatia,group stage


### Computiong the group satage Ranking

In [0]:
import pandas as pd
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("example-app") \
    .getOrCreate()

# Assuming you have a DataFrame named ranking_group obtained from your SQL table
# Replace the SQL query with your actual SQL query to fetch data from the ranking_group table
ranking_group = spark.sql("SELECT * FROM ranking_group")

# Convert the DataFrame to a Pandas DataFrame
ranking_group_pandas = ranking_group.toPandas()

# Create visualize_table DataFrame with 'teamname' column dynamically
visualize_table = pd.DataFrame({
    'teamname': ranking_group_pandas['TeamName'],
    'teamgroup': ranking_group_pandas['TeamGroup'],
    'rankings': ranking_group_pandas['Rankings'],
    'points': ranking_group_pandas['Points'],
    'goaldifference': ranking_group_pandas['GoalDifference']
})

# Create a DataFrame with blank rows
blank_rows = pd.DataFrame({'teamname': [''] * 1, 'teamgroup': [''] * 1, 'rankings': [''] * 1, 'points': [''] * 1, 'goaldifference': [''] * 1})

# Initialize an empty list to store chunks of data
chunks = []

# Split the visualize_table DataFrame into chunks of 4 records
for i in range(0, len(visualize_table), 4):
    chunk = visualize_table.iloc[i:i+4]  # Get a chunk of 4 records
    chunks.append(chunk)  # Append the chunk to the list
    if i < len(visualize_table) - 4:
        chunks.append(blank_rows)  # Append 2 blank rows after each chunk, except for the last chunk

# Concatenate all chunks to create the final DataFrame
visualize_table_with_blanks = pd.concat(chunks).reset_index(drop=True)
visualize_table_with_blanks = visualize_table_with_blanks.append(blank_rows, ignore_index = True)
half_length = len(visualize_table_with_blanks) // 2

# Create visualize_quater_final1 DataFrame with 4 blank rows at the beginning
visualize_table_with_blanks1 = pd.DataFrame(columns=['teamname','teamgroup','rankings','points','goaldifference'])
visualize_table_with_blanks1 = visualize_table_with_blanks1.append(blank_rows, ignore_index = True)
visualize_table_with_blanks1 = pd.concat([visualize_table_with_blanks1, visualize_table_with_blanks.iloc[:half_length]]).reset_index(drop=True)
#visualize_table_with_blanks1 = visualize_table_with_blanks1.drop(visualize_table_with_blanks1.tail(2).index)
visualize_table_with_blanks1['space'] = ''
visualize_table_with_blanks2 = pd.DataFrame(columns=['teamname','teamgroup','rankings','points','goaldifference'])
visualize_table_with_blanks2 = visualize_table_with_blanks2.append(blank_rows, ignore_index = True)
visualize_table_with_blanks2 = pd.concat([visualize_table_with_blanks2, visualize_table_with_blanks.iloc[half_length:]]).reset_index(drop=True)



<command-1469189203607398>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  visualize_table_with_blanks = visualize_table_with_blanks.append(blank_rows, ignore_index = True)
<command-1469189203607398>:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  visualize_table_with_blanks1 = visualize_table_with_blanks1.append(blank_rows, ignore_index = True)
<command-1469189203607398>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  visualize_table_with_blanks2 = visualize_table_with_blanks2.append(blank_rows, ignore_index = True)


### Showing the Groupstage rankgin along with all the Eurocup Knockout Fixtures

In [0]:
#grup stage table view
import pandas as pd
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("example-app") \
    .getOrCreate()
round_of_16 = spark.sql("SELECT * FROM matchesofcountry WHERE stage = 'Roundof 16' order by MatchId")
round_of_16_pandas = round_of_16.toPandas()
visualize_table16 = pd.DataFrame(columns=['Roundof16'])
visualize_table16 = visualize_table16.append({'Roundof16': ''}, ignore_index=True)
visualize_table16 = visualize_table16.append({'Roundof16': ''}, ignore_index=True)
for index, row in round_of_16_pandas.iterrows():
    visualize_table16 = visualize_table16.append({'Roundof16': f"{row['Team1']} ({row['Team1Score']})"}, ignore_index=True)
    visualize_table16 = visualize_table16.append({'Roundof16': f"{row['Team2']} ({row['Team2Score']})"}, ignore_index=True)
    
    # Insert 2 blank rows after every record
    visualize_table16 = visualize_table16.append({'Roundof16': ''}, ignore_index=True)
    visualize_table16 = visualize_table16.append({'Roundof16': ''}, ignore_index=True)
visualize_table16['space'] = ''

split_index = len(visualize_table16) // 2 - 1

# Split the final DataFrame into two halves
visualize_table16pool1 = visualize_table16.iloc[:split_index].reset_index(drop=True)
visualize_table16pool2 = visualize_table16.iloc[split_index:].reset_index(drop=True)
visualize_table16pool2 = visualize_table16pool2.drop(visualize_table16pool2.tail(2).index)

#Quater final score
quater_final_score = spark.sql("SELECT * FROM matchesofcountry WHERE stage = 'Quater Final' ORDER BY MatchId")
quater_final_score_pandas = quater_final_score.toPandas()
visualize_score16 = pd.DataFrame(columns=['teamscore'])
for i in range(4):
    visualize_score16 = visualize_score16.append({'teamscore': ''}, ignore_index=True)
for index, row in quater_final_score_pandas.iterrows():
    # Insert two records
    visualize_score16 = visualize_score16.append({'teamscore': row['Team1Score']}, ignore_index=True)
    visualize_score16 = visualize_score16.append({'teamscore': row['Team2Score']}, ignore_index=True)
    for i in range(6):
        visualize_score16 = visualize_score16.append({'teamscore': ''}, ignore_index=True)
visualize_score16 = visualize_score16.drop(visualize_score16.tail(3).index)

# quater final table view
quater_final = spark.sql("SELECT * FROM matchesofcountry WHERE stage = 'Roundof 16' ORDER BY MatchId")
quater_final_pandas = quater_final.toPandas()
visualize_quater_final = pd.DataFrame(columns=['quaterFinal'])
for i in range(4):
    visualize_quater_final = visualize_quater_final.append({'quaterFinal': ''}, ignore_index=True)
for index, row in quater_final_pandas.iterrows():
    # Insert two records
    visualize_quater_final = visualize_quater_final.append({'quaterFinal': row['Result']}, ignore_index=True)
    
    # Insert 2 blank rows after every two records
    if (index + 1) % 2 == 0:
        for i in range(6):
            visualize_quater_final = visualize_quater_final.append({'quaterFinal': ''}, ignore_index=True)
visualize_quater_final = visualize_quater_final.drop(visualize_quater_final.tail(3).index)

# Concatenate the DataFrames
visualize_quater_final = pd.concat([visualize_quater_final, visualize_score16], axis=1)

visualize_quater_final['space'] = ''

# Split the DataFrame into two halves
half_length = len(visualize_quater_final) // 2

visualize_quater_final1 = pd.DataFrame(columns=['quaterFinal'])
visualize_quater_final1 = pd.concat([visualize_quater_final1, visualize_quater_final.iloc[:half_length]]).reset_index(drop=True)

visualize_quater_final2 = pd.DataFrame(columns=['quaterFinal'])
visualize_quater_final2 = pd.concat([visualize_quater_final2, visualize_quater_final.iloc[half_length:]]).reset_index(drop=True)
visualize_quater_final2 = visualize_quater_final2.drop(visualize_quater_final2.tail(1).index)

# semifinal score
semi_final_score = spark.sql("SELECT * FROM matchesofcountry WHERE stage = 'Semi Final' ORDER BY MatchId")
semi_final_score_pandas = semi_final_score.toPandas()
visualize_score_semi = pd.DataFrame(columns=['teamscoreSF'])
for i in range(8):
    visualize_score_semi = visualize_score_semi.append({'teamscoreSF': ''}, ignore_index=True)
for index, row in semi_final_score_pandas.iterrows():
    visualize_score_semi = visualize_score_semi.append({'teamscoreSF': row['Team1Score']}, ignore_index=True)
    visualize_score_semi = visualize_score_semi.append({'teamscoreSF': row['Team2Score']}, ignore_index=True)
    for i in range(2):
        visualize_score_semi = visualize_score_semi.append({'teamscoreSF': ''}, ignore_index=True)
for i in range(6):
        visualize_score_semi = visualize_score_semi.append({'teamscoreSF': ''}, ignore_index=True)

# semifinal table view

semi_final = spark.sql("SELECT * FROM matchesofcountry WHERE stage = 'Quater Final' ORDER BY MatchId")
semi_final_pandas = semi_final.toPandas()
visualize_semi_final = pd.DataFrame(columns=['semiFinal'])
for i in range(8):
    visualize_semi_final = visualize_semi_final.append({'semiFinal': ''}, ignore_index=True)

# Iterate over each row in quater_final_pandas DataFrame
for index, row in semi_final_pandas.iterrows():
    # Insert two records
    visualize_semi_final = visualize_semi_final.append({'semiFinal': row['Result']}, ignore_index=True)
    
    # Insert 2 blank rows after every two records
    if (index + 1) % 2 == 0:
        visualize_semi_final = visualize_semi_final.append({'semiFinal': ''}, ignore_index=True)
        visualize_semi_final = visualize_semi_final.append({'semiFinal': ''}, ignore_index=True)
for i in range(6):
    visualize_semi_final = visualize_semi_final.append({'semiFinal': ''}, ignore_index=True)
visualize_semi_final = pd.concat([visualize_semi_final, visualize_score_semi], axis=1)
visualize_semi_final['space'] = ''
# Split the DataFrame into two halves
half_length = len(visualize_semi_final) // 2

# Create visualize_quater_final1 DataFrame with 4 blank rows at the beginning
visualize_semi_final1 = pd.DataFrame(columns=['semiFinal'])
visualize_semi_final1 = pd.concat([visualize_semi_final1, visualize_semi_final.iloc[:half_length]]).reset_index(drop=True)
for i in range(5):
    visualize_semi_final1 = visualize_semi_final1.append({'semiFinal': '', 'teamscoreSF': '', 'space': ''}, ignore_index=True)
# Create visualize_quater_final2 DataFrame with 4 blank rows at the beginning
visualize_semi_final2 = pd.DataFrame({'semiFinal': [''] * 7, 'teamscoreSF': [''] * 7,'space': [''] * 7})
visualize_semi_final2 = pd.concat([visualize_semi_final2, visualize_semi_final.iloc[half_length:]]).reset_index(drop=True)
visualize_semi_final2 = visualize_semi_final2.drop(visualize_semi_final2.tail(2).index)

# final table view
final = spark.sql("SELECT * FROM matchesofcountry WHERE stage = 'Final' ORDER BY MatchId")
third = spark.sql("SELECT * FROM matchesofcountry WHERE stage = 'Third Place' ORDER BY MatchId")

# Convert the DataFrame to a Pandas DataFrame
final_pandas = final.toPandas()
third_pandas = third.toPandas()

# Create a new DataFrame to store the teamnames
visualize_final = pd.DataFrame(columns=['Finals'])
for i in range(6):
    visualize_final = visualize_final.append({'Finals': ''}, ignore_index=True)

# Iterate over each row in quater_final_pandas DataFrame
for index, row in final_pandas.iterrows():
    visualize_final = visualize_final.append({'Finals': f"{row['Team1']} ({row['Team1Score']})"}, ignore_index=True)
    visualize_final = visualize_final.append({'Finals': f"{row['Team2']} ({row['Team2Score']})"}, ignore_index=True)
    visualize_final = visualize_final.append({'Finals': f"Winner: {row['Result']}"}, ignore_index=True)
    
    # Insert 2 blank rows after every two records
    if (index + 1) % 2 == 0:
        visualize_final = visualize_final.append({'Finals': ''}, ignore_index=True)
        visualize_final = visualize_final.append({'Finals': ''}, ignore_index=True)
    for i in range(2):
        visualize_final = visualize_final.append({'Finals': ''}, ignore_index=True)
for index, row in third_pandas.iterrows():
    # Insert two records
    visualize_final = visualize_final.append({'Finals': f"{row['Team1']} ({row['Team1Score']})"}, ignore_index=True)
    visualize_final = visualize_final.append({'Finals': f"{row['Team2']} ({row['Team2Score']})"}, ignore_index=True)
    visualize_final = visualize_final.append({'Finals': f"Third: {row['Result']}"}, ignore_index=True)
for i in range(2):
    visualize_final = visualize_final.append({'Finals': ''}, ignore_index=True)
visualize_final['space'] = ''

# mearging gropstage pool and quaterfinal
visualize_quater_final1.columns = [f"{col}q1" for col in visualize_quater_final1.columns]
merged_df = pd.concat([visualize_table16pool1, visualize_quater_final1], axis=1)

visualize_semi_final1.columns = [f"{col}s1" for col in visualize_semi_final1.columns]
merged_df = pd.concat([merged_df, visualize_semi_final1], axis=1)

visualize_final.columns = [f"{col}f" for col in visualize_final.columns]
merged_df = pd.concat([merged_df, visualize_final], axis=1)

visualize_semi_final2.columns = [f"{col}s2" for col in visualize_semi_final2.columns]

# Concatenate the DataFrames
merged_df = pd.concat([merged_df, visualize_semi_final2], axis=1)

visualize_quater_final2.columns = [f"{col}q2" for col in visualize_quater_final2.columns]

# Concatenate the DataFrames
merged_df = pd.concat([merged_df, visualize_quater_final2], axis=1)

visualize_table16pool2.columns = [f"{col}r2" for col in visualize_table16pool2.columns]

# Concatenate the DataFrames
merged_df = pd.concat([merged_df, visualize_table16pool2], axis=1)

visualize_table_with_blanks1.columns = [f"{col}g1" for col in visualize_table_with_blanks1.columns]
visualize_table_with_blanks2.columns = [f"{col}g2" for col in visualize_table_with_blanks2.columns]

# Concatenate the DataFrames
merged_df = pd.concat([visualize_table_with_blanks1,merged_df, visualize_table_with_blanks2], axis=1)

# Display the merged DataFrame
display(merged_df)

<command-1005514274519437>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  visualize_table16 = visualize_table16.append({'Roundof16': ''}, ignore_index=True)
<command-1005514274519437>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  visualize_table16 = visualize_table16.append({'Roundof16': ''}, ignore_index=True)
<command-1005514274519437>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  visualize_table16 = visualize_table16.append({'Roundof16': f"{row['Team1']} ({row['Team1Score']})"}, ignore_index=True)
<command-1005514274519437>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  visualize_table16 = visualize_table16.append({'Rou

teamnameg1,teamgroupg1,rankingsg1,pointsg1,goaldifferenceg1,spaceg1,Roundof16,space,quaterFinalq1,teamscoreq1,spaceq1,semiFinals1,teamscoreSFs1,spaces1,Finalsf,spacef,semiFinals2,teamscoreSFs2,spaces2,quaterFinalq2,teamscoreq2,spaceq2,Roundof16r2,spacer2,teamnameg2,teamgroupg2,rankingsg2,pointsg2,goaldifferenceg2
,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Scotland,A,1,9,10,,,,,,,,,,,,,,,,,,,,France,D,1,7,7
Switzerland,A,2,3,-3,,Spain (4),,,,,,,,,,,,,,,,Romania (6),,Netherlands,D,2,5,1
Hungary,A,3,3,-3,,Türkiye (3),,,,,,,,,,,,,,,,Poland (3),,Poland,D,3,4,-1
Germany,A,4,3,-4,,,,Spain,3,,,,,,,,,,Romania,4,,,,Austria,D,4,0,-7
,,,,,,,,Scotland,1,,,,,,,,,,France,1,,,,,,,,
Spain,B,1,7,5,,Scotland (6),,,,,,,,Spain (3),,,,,,,,France (5),,Romania,E,1,7,5
Croatia,B,2,7,4,,Serbia (6),,,,,,,,Romania (4),,,,,,,,Greece (4),,Belgium,E,2,6,1
Albania,B,3,3,-3,,,,,,,Spain,6,,Winner: Romania,,Romania,5,,,,,,,Ukraine,E,3,3,-3
Italy,B,4,0,-6,,,,,,,Belgium,6,,,,Croatia,3,,,,,,,Slovakia,E,4,1,-3
